# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\pedro\Desktop\Projeto1_CDados21-1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Burger King.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,queria um burger king agora https://t.co/ddr6r...,1
1,as vezes publicitários me irritam tanto q tive...,1
2,ano que vem burger king vai ta patrocinando a ...,0
3,"só um burger king, é sobre isso",0
4,minha fome tem nome específico burger king o ...,1


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,rt @_bradachi: mc donalds é melhor que burger ...,1
1,mds eu acabei de ver uma propagando no yt do b...,1
2,@burgerkingbr todas as vezes que fui no burger...,1
3,@souamorgan eu te entendo... tipo logo abaixo ...,0
4,ifood e burger king sempre deixa forte kkk 🙅💪😅,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
#LIMPADOR DE PLAVRAS V.10000000
import re 

def cleanup(text):

    punctuation = '[!-.:?;/,""''@#$%^&*()+=]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    
lista_palavras_train = []
i=0
while i < len(train.Treinamento) :
    lista_das_palavras_do_tweet_limpo = cleanup(train.Treinamento[i]).split()
    for elemento in lista_das_palavras_do_tweet_limpo:
        lista_palavras_train.append(elemento)
    i+= 1


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**